In [ ]:
import pandas as pd
import clickhouse_connect

pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.options.display.width = None
pd.set_option('display.max_rows', 100)
pd.set_option('display.min_rows', 20)

In [ ]:
# 10 x 5
states = ["Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado", "Connecticut", "Delaware", "Florida", "Georgia",
            "Hawaii", "Idaho", "Illinois", "Indiana", "Iowa", "Kansas", "Kentucky", "Louisiana", "Maine", "Maryland",
            "Massachusetts", "Michigan", "Minnesota", "Mississippi", "Missouri", "Montana", "Nebraska", "Nevada", "New Hampshire", "New Jersey",
            "New Mexico", "New York", "North Carolina", "North Dakota", "Ohio", "Oklahoma", "Oregon", "Pennsylvania", "Rhode Island", "South Carolina",
            "South Dakota", "Tennessee", "Texas", "Utah", "Vermont", "Virginia", "Washington", "West Virginia", "Wisconsin", "Wyoming"]

CLICKHOUSE_CLOUD_HOSTNAME = 'csi4142-clickhouse.mershab.xyz'
CLICKHOUSE_CLOUD_USER = 'csi'
CLICKHOUSE_CLOUD_PASSWORD = 'iceberg'

TABLE_NAME1 = 'cost_of_living_dimension_table_test'
TABLE_NAME2 = 'income_dimension_table_test'

In [ ]:
# Yearly Median Household Income (USD) by State from 1984 - 2022

df_list_income = []
single_ticker_df = {}
for state in states:
    temp_df = pd.DataFrame(pd.read_csv(f"../data/income_data/{state}Income.csv")) #, parse_dates=['datetime']
    
    temp_df.columns.values[1] = "Median Household Income in USD"
    temp_df = temp_df.rename(columns={"DATE": "Year"})

    temp_df['Year'] = pd.DatetimeIndex(temp_df['Year']).year
    temp_df['State'] = state
    df_list_income.append(temp_df) 


df_list_income

In [ ]:
# Data for Cost of Living Index runs from 1960-2003, 2016-2017, 2019-2023
# Data for Cost of Living Index unavailable for certain states prior to 2016 (Hawaii, Alaska)

# "location","indicator","Unit","Date","Value"

# Data for Indicators ("Housing", "Groceries", "Utilities", "Transportation", "Miscellaneous", "Health Care") from 2017, 2019-2023

df_list_col = []
single_ticker_df = {}

temp_df = pd.DataFrame(pd.read_csv(f"../data/cost_of_living_data/cost_of_living_data.csv"))
temp_df = temp_df.drop(columns=['Unit'])
temp_df = temp_df.rename(columns={"Date": "Year", "Value": "Cost of Living Index", "location": "State", "indicator": "Indicator"})

temp_df['Year'] = pd.DatetimeIndex(temp_df['Year']).year
df_list_col.append(temp_df) 

df_list_col

In [ ]:
df_income = pd.concat(df_list_income)

df_income

In [ ]:
df_col = pd.concat(df_list_col)

df_col

In [10]:
print(df_col.dtypes)
print(df_income.dtypes)

State                    object
Indicator                object
Year                      int32
Cost of Living Index    float64
dtype: object
Year                               int32
Median Household Income in USD     int64
State                             object
dtype: object


In [ ]:
client = clickhouse_connect.get_client(host=CLICKHOUSE_CLOUD_HOSTNAME, port=443, user=CLICKHOUSE_CLOUD_USER, password=CLICKHOUSE_CLOUD_PASSWORD,database='default')

print(client.ping())
print("connected to " + CLICKHOUSE_CLOUD_HOSTNAME + "\n")

In [16]:
# Check if table exists and create if it doesn't
create_table_query = f"""
CREATE TABLE IF NOT EXISTS {TABLE_NAME2} (
        State         String,
        Indicator     String,
        Year          String,
        Index         Float64
) ENGINE = MergeTree()
ORDER by Year
"""

#client.command(f'DROP TABLE IF EXISTS {TABLE_NAME}')
client.command(create_table_query)

DatabaseError: :HTTPDriver for https://csi4142-clickhouse.mershab.xyz:443 returned response code 400)
 Code: 62. DB::Exception: Syntax error: failed at position 179 (')') (line 7, col 1): ) ENGINE = MergeTree()
. Expected one of: lambda expression, CAST operator, NOT, INTERVAL, CASE, DATE, TIMESTAMP, tuple, collection of literals, array, number, literal, NULL, Bool, true, false, string literal, asterisk, qualified asterisk, compound identifier, list of elements, identifier, COLUMNS matcher, COLUMNS, qualified COLUMNS matcher, substitution, MySQL-style global variable. (SYNTAX_ERROR) (version 24.2.2.71 (official build))


In [14]:
create_table_query = f"""
CREATE TABLE IF NOT EXISTS {TABLE_NAME2} (
        Year                               UInt32,
        Median Household Income in USD     UInt64,
        State                             String
) ENGINE = MergeTree()
ORDER BY State
"""

client.command(create_table_query)

DatabaseError: :HTTPDriver for https://csi4142-clickhouse.mershab.xyz:443 returned response code 400)
 Code: 62. DB::Exception: Syntax error: failed at position 135 ('Income') (line 4, col 26): Income in USD     UInt64,
        State                             String
) ENGINE = MergeTree()
ORDER BY State
. Expected one of: COLLATE, NOT, NULL, DEFAULT, MATERIALIZED, ALIAS, EPHEMERAL, AUTO_INCREMENT, COMMENT, CODEC, STATISTIC, TTL, PRIMARY KEY, SETTINGS, token, Comma, ClosingRoundBracket. (SYNTAX_ERROR) (version 24.2.2.71 (official build))
